In [10]:
from crewai import Agent, Task, Crew, LLM
from langchain_community.llms import Ollama
from dotenv import load_dotenv
import os

load_dotenv()

# CrewAI LLM 클래스를 사용하여 Ollama 설정
llm = LLM(
    model="ollama/exaone3.5:2.4b",
    base_url="http://localhost:11434"
)

In [11]:
# Agents
planner = Agent(
    role="콘텐츠 기획자",
    goal="""
    주제 {topic}에 대해 흥미롭고 사실에 기반한 콘텐츠 기획안을 작성합니다.
    독자들이 유익한 정보를 얻고 올바른 결정을 내릴 수 있도록 돕는 것이 목표입니다.
    모든 결과는 반드시 한국어로 작성합니다.
    """,
    backstory="""
    당신은 Medium(https://medium.com/)에 게시될 블로그 글의 기획을 담당하고 있습니다.
    독자를 고려한 콘텐츠 구조를 설계하고,
    이후 콘텐츠 작가가 바로 집필할 수 있도록
    명확하고 실용적인 기획 문서를 제공합니다.
    """,
    llm=llm,
    allow_delegation=False,
    verbose=True
)

writer = Agent(
    role="콘텐츠 작가",
    goal="""
    주제 {topic}에 대한 통찰력 있고 사실에 기반한 블로그 글을 작성합니다.
    콘텐츠 기획자가 제공한 개요를 충실히 따르며,
    의견과 객관적 사실을 명확히 구분합니다.
    모든 결과는 반드시 한국어로 작성합니다.
    """,
    backstory="""
    당신은 Medium에 게시될 블로그 글을 집필하는 콘텐츠 작가입니다.
    기획자가 제공한 구조와 핵심 메시지를 바탕으로,
    독자가 끝까지 읽고 신뢰할 수 있는 글을 작성합니다.
    """,
    llm=llm,
    allow_delegation=False,
    verbose=True
)

editor = Agent(
    role="편집자",
    goal="""
    완성된 블로그 글을 Medium 스타일에 맞게 편집합니다.
    문법, 흐름, 표현을 다듬고
    과도한 주장이나 편향된 표현이 없는지 점검합니다.
    모든 결과는 반드시 한국어로 작성합니다.
    """,
    backstory="""
    당신은 콘텐츠 작가가 작성한 원고를 검토하는 편집자입니다.
    글의 명확성, 일관성, 중립성을 기준으로
    출판 가능한 최종 원고를 완성합니다.
    """,
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# Tasks
plan = Task(
    description="""
    1. 주제 {topic}와 관련된 최신 트렌드와 핵심 개념을 정리합니다.
    2. 대상 독자를 정의하고, 그들의 관심사와 문제점을 분석합니다.
    3. 서론, 본문 구조, 결론을 포함한 상세 콘텐츠 개요를 작성합니다.
    4. SEO 키워드와 참고할 만한 자료 방향을 제안합니다.
    """,
    expected_output="""
    - 대상 독자 분석
    - 전체 콘텐츠 구조(섹션 단위 개요)
    - 핵심 메시지
    - SEO 키워드 목록
    """,
    agent=planner
)

write = Task(
    description="""
    콘텐츠 기획 문서를 바탕으로 주제 {topic}에 관한 블로그 글을 작성합니다.
    각 섹션은 명확한 소제목을 포함하며,
    서론–본문–결론 구조를 갖추도록 합니다.
    """,
    expected_output="""
    마크다운 형식의 완성된 블로그 글
    (각 섹션당 2~3단락 분량)
    """,
    agent=writer
)

edit = Task(
    description="""
    작성된 블로그 글을 검토하고 편집합니다.
    문법 오류, 어색한 표현, 논리 흐름을 개선하고
    출판 가능한 최종 원고를 완성합니다.
    """,
    expected_output="""
    출판 준비가 완료된 최종 블로그 글 (Markdown)
    """,
    agent=editor
)


In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],  
    verbose=True
)

In [ ]:
inputs = {
    "topic": "LangGraph, AutoGen, CrewAI를 활용한 멀티 에이전트 시스템 구축 비교 연구"
}

result = crew.kickoff(inputs=inputs)
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0d482145-963c-47b7-9aa7-65b756c6b34f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 콘텐츠 기획자                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│      1. 주제 LangGraph, AutoGen, CrewAI를 활용한 멀티 에이전트 시스템 구축 비교 연구와 관련된 최신 트렌드와     │
│  핵심 개념을 정리합니다.                                                                                        │
│      2. 대상 독자를 정의하고, 그들의 관심사와 문제점을 분석합니다.                                              │
│      3. 서론, 본문 구조, 결론을 포함한 상세 콘텐츠 개요를 작성합니다.                                           │
│      4. SEO 키워드와 참고할 만한 자료 방향을 제안합니다.                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 콘텐츠 기획자                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  #### 대상 독자 분석:                                                                                           │
│  - **주요 독자**: 기술 혁신에 관심이 많은 IT 전문가, 스타트업 창업자, 연구원 및 엔지니어                        │
│  - **관심사**:                                                                                                  │
│    - AI 및 머신러닝 기술의 최신 트렌드 이해                                                                     │
│    - 멀티 에이전트 시스템 구축 방법론과 실제 적용 사례 탐색                                                     │
│    - 다양한 AI 도구(LangGraph, AutoGen, CrewAI)의 기능 비교 및 효과 분석                                        │
│  - **문제점**:                                                                                                  │
│    - 복잡한 멀티 에이전트 시스템 설계 및 운영에서 발생하는 기술적 난관 해결 방법 부족                           │
│    - 효율적인 자원 배분 및 최적화 전략 부재                                                                     │
│    - 다양한 AI 도구 간의 선택 기준 및 통합 방법에 대한 명확한 가이드라인 필요                                   │
│                                                                                                                 │
│  #### 전체 콘텐츠 구조(섹션 단위 개요):                                                                         │
│                                                                                                                 │
│  1. **서론**                                                                                                    │
│     - **개요**: 멀티 에이전트 시스템의 중요성과 최근 발전 동향 소개                                             │
│     - **주제 소개**: LangGraph, AutoGen, CrewAI의 역할과 핵심 기능 간략히 설명                                  │
│     - **연구 목적**: 비교 연구를 통해 독자들이 효과적인 멀티 에이전트 시스템 구축 전략 수립에 도움 주기         │
│                                                                                                                 │
│  2. **기술 개요 및 배경**                                                                                       │
│     - **LangGraph**: 자연어 처리(NLP) 측면에서의 역할과 특징                                                    │
│     - **AutoGen**: 자동화 생성 기술의 핵심 개념 및 활용 사례                                                    │
│     - **CrewAI**: 협업 및 멀티 에이전트 관리의 혁신적 접근법                                                    │
│     - **비교 프레임워크**: 세 시스템 간의 기능적 차이점과 상호 보완성 강조                                      │
│                                                                                                                 │
│  3. **실제 적용 사례 분석**                                                                                     │
│     - **사례 연구 1**: LangGraph 기반 NLP 프로젝트 성공 사례 분석                                               │
│     - **사례 연구 2**: AutoGen을 활용한 자동화 프로세스 최적화 사례                                             │
│     - **사례 연구 3**: CrewAI의 협업 환경 구축 성공 사례 및 도전 과제                                           │
│                                                                                                                 │
│  4. **비교 연구 방법론**                                                                                        │
│     - **연구 접근 방식**: 실험 설계, 데이터 수집 및 분석 방법 설명                                              │
│     - **평가 지표**: 효율성, 통합 용이성, 성능 최적화 측면에서의 평가 기준 제시                                 │
│     - **결과 요약**: 각 시스템의 강점과 약점 비교 결과 공개                                                     │
│                                                                                              

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9c48cb3b-189a-4697-b320-0f9c2d512c66                                                                     │
│  Agent: 콘텐츠 기획자                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 콘텐츠 작가                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  LangGraph, AutoGen, CrewAI 기반 멀티 에이전트 시스템 구축 비교 연구 블로그 글                                  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### 서론                                                                                                      │
│                                                                                                                 │
│  **멀티 에이전트 시스템의 중요성과 최근 동향**                                                                  │
│                                                                                                                 │
│  기술의 발전은 복잡한 시스템 관리와 자동화 과정에서 멀티 에이전트 시스템의 필요성을 더욱 부각시켰습니다. 이     │
│  시스템은 다양한 자율적 에이전트가 상호작용하며 공동 목표를 달성하는 구조로, 특히 인공지능 분야에서 그          │
│  중요성이 빠르게 증대하고 있습니다. 최근에는 자연어 처리(NLP), 자동화 생성, 그리고 고급 협업 관리 기술이        │
│  결합되어 더욱 복잡하고 효율적인 시스템 구축이 가능해졌습니다. 본 연구는 **LangGraph**, **AutoGen**,            │
│  **CrewAI** 세 가지 주요 AI 도구를 중심으로 그 역할과 효과를 비교 분석함으로써, 독자들이 최적의 멀티 에이전트   │
│  시스템 구축 전략을 수립하는 데 필요한 통찰력을 제공하고자 합니다.                                              │
│                                                                                                                 │
│  **주제 소개**                                                                                                  │
│                                                                                                                 │
│  - **LangGraph**: 자연어 이해와 생성 능력을 통해 에이전트 간의 언어 기반 상호작용을 지원합니다.                 │
│  - **AutoGen**: 자동화 프로세스 최적화를 위한 알고리즘을 제공하여 반복적 작업의 효율성을 극대화합니다.          │
│  - **CrewAI**: 에이전트 간의 협업과 동기화를 혁신적으로 지원하며, 복잡한 환경에서의 효율적인 작업 분배를        │
│  가능하게 합니다.                                                                                               │
│                                                                                                                 │
│  **연구 목적**                                                                                                  │
│                                                                                                                 │
│  본 연구는 각 도구의 기능적 강점과 한계를 명확히 파악하여 IT 전문가, 스타트업 창업자, 연구원 등이 실질적인      │
│  구축 전략을 세울 수 있도록 돕는 것을 목표로 합니다. 이를 통해 기술적 난관을 해결하고 효율적인 자원 관리 및     │
│  최적화를 실현할 수 있는 지침을 제시합니다.                                                                     │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### 기술 개요 및 배경                                                                                         │
│                                                                                                                 │
│  **LangGraph**                                                                                                  │
│                                                                                                         